In [1]:
import scipy.io as sio
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

In [3]:
data = sio.loadmat("D:\Masters\sem2\gnr602/nn_bagging\Indian_pines_corrected.mat")
labels = sio.loadmat("D:\Masters\sem2\gnr602/nn_bagging\Indian_pines_gt.mat")
data = data[sorted(data.keys())[-1]]
labels = labels[sorted(labels.keys())[-1]]

In [4]:
data_reshaped = np.reshape(data, (-1, data.shape[2]))

# Preprocess the data using PCA
pca = PCA(n_components=30)
data_pca = pca.fit_transform(data_reshaped)

# Reshape labels
labels_reshaped = np.reshape(labels, (-1,))

In [5]:
nonzero_labels_idx = np.where(labels_reshaped != 0)[0]
data_pca = data_pca[nonzero_labels_idx]
labels_reshaped = labels_reshaped[nonzero_labels_idx]

# Split data into training and testing sets
train_idx = np.random.choice(data_pca.shape[0], int(0.7 * data_pca.shape[0]), replace=False)
test_idx = np.setdiff1d(np.arange(data_pca.shape[0]), train_idx)

X_train, y_train = data_pca[train_idx], labels_reshaped[train_idx]
X_test, y_test = data_pca[test_idx], labels_reshaped[test_idx]

In [6]:
# Train SVM with RBF kernel using one-vs-all method
clf_ova_rbf = OneVsRestClassifier(SVC(kernel='rbf', C=1, gamma=0.1))
clf_ova_rbf.fit(X_train, y_train)

OneVsRestClassifier(estimator=SVC(C=1, gamma=0.1))

In [7]:
# Test SVM with RBF kernel using one-vs-all method
y_pred_ova_rbf = clf_ova_rbf.predict(X_test)

In [8]:
accuracy_ova_rbf = accuracy_score(y_test, y_pred_ova_rbf)
print("Accuracy (one-vs-one, RBF kernel):", accuracy_ova_rbf)

Accuracy (one-vs-one, RBF kernel): 0.24


In [10]:
# Import necessary libraries
from sklearn.datasets import load_iris
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.spatial import distance
import numpy as np

# Load iris dataset
iris = load_iris()

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Apply Linear Discriminant Analysis to reduce dimensionality
lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

# Train a binary SVM classifier for each class of the reduced data
svm_models = []
for i in np.unique(y_train):
    y_train_binary = np.zeros(y_train.shape)
    y_train_binary[y_train == i] = 1
    svm = SVC(kernel='linear')
    svm.fit(X_train_lda, y_train_binary)
    svm_models.append(svm)

# Assign a score to each test point for each class using the trained SVM classifiers
scores = np.zeros((X_test_lda.shape[0], len(np.unique(y_train))))
for i, svm in enumerate(svm_models):
    scores[:, i] = svm.decision_function(X_test_lda)

# Calculate the similarity between the test point and each class using the score of the SVM classifier and the distance to the nearest neighbor of that class in the reduced data
similarity_scores = np.zeros((X_test_lda.shape[0], len(np.unique(y_train))))
for i in range(X_test_lda.shape[0]):
    for j, svm in enumerate(svm_models):
        dist = distance.euclidean(X_test_lda[i], X_train_lda[y_train == j][np.argmin(distance.cdist([X_test_lda[i]], X_train_lda[y_train == j]))])
        similarity_scores[i, j] = scores[i, j] / dist

# Assign the test point to the class with the highest similarity score
predicted = np.argmax(similarity_scores, axis=1)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, predicted)

print("Accuracy:", accuracy)


Accuracy: 0.9


In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
import os,time
from sklearn.multiclass import  OneVsRestClassifier
from sklearn.metrics import accuracy_score

data = sio.loadmat("D:\Masters\sem2\gnr602/nn_bagging\Indian_pines_corrected.mat")
labels = sio.loadmat("D:\Masters\sem2\gnr602/nn_bagging\Indian_pines_gt.mat")
data = data[sorted(data.keys())[-1]]
labels = labels[sorted(labels.keys())[-1]]

def applyPCA(X, n_components=30, seed=1):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=n_components, whiten=True, random_state=seed)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], n_components))
    return newX

# data.shape

data = applyPCA(data)
# data.shape

# labels = gt_data[sorted(gt_data.keys())[-1]]





data = data.reshape(data.shape[0]*data.shape[1],data.shape[2])
data.shape

labels = labels.reshape(-1,1)
nonzero_labels_idx = np.where(labels != 0)[0]
data = data[nonzero_labels_idx]
labels = labels[nonzero_labels_idx]
print(data.shape)
print(labels.shape)

combine = np.concatenate((data,labels),axis=1)
combine

df = pd.DataFrame(combine)

# df

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

C_values = [0.1, 1, 10, 100]
kernel_type = ['linear','poly','rbf']
accuracy = []

for kernel in kernel_type:
  for C in C_values:
      # Initialize SVM classifier with current C value
      svm = SVC(kernel=kernel, C=C)

      start_time1 = time.time()
      svm_ovr = OneVsRestClassifier(svm).fit(X_train, y_train)
      Time_taken_ovr = time.time() - start_time1
      y_test_ovr = svm_ovr.predict(X_test)

      acc_ovr = accuracy_score(y_test, y_test_ovr)
      accuracy.append(acc_ovr)

      
      # # Train SVM classifier
      # svm.fit(X_train, y_train)
      
      # # Get number of support vectors
      # n_support_vectors = svm.n_support_
      
      # # Print number of support vectors for current C value
      # print(f"C = {C}: Number of support vectors = {sum(n_support_vectors)}")

(10249, 30)
(10249, 1)


In [2]:
# -*- coding: utf-8 -*-
"""ML_SVM.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Xa88GcUIDKDUFGXIkLbVu6ynsc8qj0nT
"""

import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
import os,time
from sklearn.multiclass import  OneVsRestClassifier
from sklearn.metrics import accuracy_score

hyp_data = sio.loadmat("D:\Masters\sem2\gnr602/nn_bagging\Indian_pines_corrected.mat")
gt_data = sio.loadmat("D:\Masters\sem2\gnr602/nn_bagging\Indian_pines_gt.mat")

data = hyp_data[sorted(hyp_data.keys())[-1]]

def applyPCA(X, n_components=30, seed=1):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=n_components, whiten=True, random_state=seed)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], n_components))
    return newX

data.shape

data = applyPCA(data)
data.shape

labels = gt_data[sorted(gt_data.keys())[-1]]

print(data.shape)
print(labels.shape)

data = data.reshape(data.shape[0]*data.shape[1],data.shape[2])
data.shape

labels = labels.reshape(-1,1)
labels.shape

unique, counts = np.unique(labels, return_counts=True)

print(unique,counts)

count_sort_ind = np.argsort(-counts)
print(unique[count_sort_ind])
counts[count_sort_ind]

data.shape

combine = np.concatenate((data,labels),axis=1)
combine

combine = np.delete(combine,np.where(combine[:,-1]==0),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==1),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==2),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==4),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==5),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==7),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==8),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==9),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==11),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==12),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==13),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==15),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==16),axis=0)

combine.shape

df = pd.DataFrame(combine)

df

data = np.array(combine)[:,:-1]
labels = np.array(combine)[:,-1]

print(data.shape)
print(labels.shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

C_values = [0.1, 1, 10, 100]
kernel_type = ['linear','poly','rbf']
accuracy = []

for kernel in kernel_type:
  for C in C_values:
      # Initialize SVM classifier with current C value
      svm = SVC(kernel=kernel, C=C)

      start_time1 = time.time()
      svm_ovr = OneVsRestClassifier(svm).fit(X_train, y_train)
      Time_taken_ovr = time.time() - start_time1
      y_test_ovr = svm_ovr.predict(data)

      acc_ovr = accuracy_score(labels, y_test_ovr)
      accuracy.append(acc_ovr)

      
      # # Train SVM classifier
      # svm.fit(X_train, y_train)
      
      # # Get number of support vectors
      # n_support_vectors = svm.n_support_
      
      # # Print number of support vectors for current C value
      # print(f"C = {C}: Number of support vectors = {sum(n_support_vectors)}")

print(accuracy)

(145, 145, 30)
(145, 145)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] [10776    46  1428   830   237   483   730    28   478    20   972  2455
   593   205  1265   386    93]
[ 0 11  2 14 10  3  6 12  5  8 15  4 13 16  1  7  9]
(3797, 30)
(3797,)
[0.9317882538846458, 0.9399525941532789, 0.9436397155649197, 0.9446931788253885, 0.9017645509612853, 0.9483803002370292, 0.9841980510929681, 0.9902554648406637, 0.9020279167764024, 0.9678693705557019, 0.9894653673953121, 0.9928891229918356]
